In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install mediapipe opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 41.5 MB/s eta 0:00:00


In [4]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import os
from pathlib import Path

# Setup mediapipe instance for hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Define all hand landmarks with their names
HAND_LANDMARKS_DICT = {
    'THUMB_CMC': mp_hands.HandLandmark.THUMB_CMC,
    'THUMB_MCP': mp_hands.HandLandmark.THUMB_MCP,
    'THUMB_IP': mp_hands.HandLandmark.THUMB_IP,
    'THUMB_TIP': mp_hands.HandLandmark.THUMB_TIP,
    'INDEX_FINGER_MCP': mp_hands.HandLandmark.INDEX_FINGER_MCP,
    'INDEX_FINGER_PIP': mp_hands.HandLandmark.INDEX_FINGER_PIP,
    'INDEX_FINGER_DIP': mp_hands.HandLandmark.INDEX_FINGER_DIP,
    'INDEX_FINGER_TIP': mp_hands.HandLandmark.INDEX_FINGER_TIP,
    'MIDDLE_FINGER_MCP': mp_hands.HandLandmark.MIDDLE_FINGER_MCP,
    'MIDDLE_FINGER_PIP': mp_hands.HandLandmark.MIDDLE_FINGER_PIP,
    'MIDDLE_FINGER_DIP': mp_hands.HandLandmark.MIDDLE_FINGER_DIP,
    'MIDDLE_FINGER_TIP': mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
    'RING_FINGER_MCP': mp_hands.HandLandmark.RING_FINGER_MCP,
    'RING_FINGER_PIP': mp_hands.HandLandmark.RING_FINGER_PIP,
    'RING_FINGER_DIP': mp_hands.HandLandmark.RING_FINGER_DIP,
    'RING_FINGER_TIP': mp_hands.HandLandmark.RING_FINGER_TIP,
    'PINKY_MCP': mp_hands.HandLandmark.PINKY_MCP,
    'PINKY_PIP': mp_hands.HandLandmark.PINKY_PIP,
    'PINKY_DIP': mp_hands.HandLandmark.PINKY_DIP,
    'PINKY_TIP': mp_hands.HandLandmark.PINKY_TIP
}

def add_title_bar(image, title, bar_height=60, bg_color=(245, 117, 16)):
    """Add a title bar to the top of the image"""
    h, w = image.shape[:2]
    title_bar = np.full((bar_height, w, 3), bg_color, dtype=np.uint8)

    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1
    font_thickness = 2
    text_color = (255, 255, 255)

    text_size = cv2.getTextSize(title, font, font_scale, font_thickness)[0]
    text_x = (w - text_size[0]) // 2
    text_y = (bar_height + text_size[1]) // 2

    cv2.putText(title_bar, title, (text_x, text_y), font, font_scale, text_color, font_thickness)
    return np.vstack((title_bar, image))

def get_video_files(master_folder):
    """Get the first video file from each subdirectory"""
    video_files = []
    video_extensions = ('.mp4', '.avi', '.mov', '.mkv')

    for folder_path in Path(master_folder).iterdir():
        if folder_path.is_dir():
            # Get all video files in the current folder
            folder_videos = [f for f in folder_path.iterdir() if f.suffix.lower() in video_extensions]
            if folder_videos:
                # Sort videos to ensure consistent selection and take the first one
                first_video = sorted(folder_videos)[0]
                sign_name = folder_path.name
                video_files.append((str(first_video), sign_name))

    return video_files

def process_video(input_path, sign_name, output_folder):
    """Process a single video file"""
    # Create output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Generate output path
    output_filename = f"{sign_name}_landmarks.mp4"
    output_path = os.path.join(output_folder, output_filename)

    # Initialize VideoCapture
    cap = cv2.VideoCapture(input_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    # Calculate dimensions for the frame with title bar
    title_bar_height = 60
    combined_height = height + title_bar_height

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, combined_height))

    # Setup mediapipe hands instance
    with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=2,
        min_detection_confidence=0.7,
        min_tracking_confidence=0.5
    ) as hands:
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        current_frame = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            current_frame += 1
            if current_frame % 10 == 0:
                print(f"Processing {sign_name}: frame {current_frame} of {frame_count} ({(current_frame/frame_count*100):.1f}%)")

            # Convert BGR to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Process the image
            results = hands.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:
                    # Draw hand connections
                    mp_drawing.draw_landmarks(
                        image,
                        hand_landmarks,
                        mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style()
                    )

                    # Add landmark points
                    for landmark_id in HAND_LANDMARKS_DICT.values():
                        landmark = hand_landmarks.landmark[landmark_id]
                        x = int(landmark.x * width)
                        y = int(landmark.y * height)
                        cv2.circle(image, (x, y), 4, (0, 255, 0), -1)

            # Add title bar with sign name
            video_title = f"Sign Language - {sign_name} (Hand Landmarks)"
            final_image = add_title_bar(image, video_title)

            # Write the frame
            out.write(final_image)

        cap.release()
        out.release()
        print(f"Completed processing {sign_name}! Output saved to: {output_path}")

def main():
    # Define master folder path and output folder
    master_folder = "/content/drive/MyDrive/sign_language_project/THIWANKA 2/"
    output_folder = "/content/drive/MyDrive/sign_language_project/processed_videos/"

    # Get list of videos to process
    video_files = get_video_files(master_folder)

    print(f"Found {len(video_files)} videos to process")

    # Process each video
    for input_path, sign_name in video_files:
        print(f"\nProcessing sign: {sign_name}")
        print(f"Input video: {input_path}")
        process_video(input_path, sign_name, output_folder)

    print("\nAll videos processed successfully!")

if __name__ == "__main__":
    main()

Found 17 videos to process

Processing sign: BUTTERFLY
Input video: /content/drive/MyDrive/sign_language_project/THIWANKA 2/BUTTERFLY/WhatsApp Video 2024-10-23 at 13.03.12_9c7463b6 - Trim.mp4
Processing BUTTERFLY: frame 10 of 114 (8.8%)
Processing BUTTERFLY: frame 20 of 114 (17.5%)
Processing BUTTERFLY: frame 30 of 114 (26.3%)
Processing BUTTERFLY: frame 40 of 114 (35.1%)
Processing BUTTERFLY: frame 50 of 114 (43.9%)
Processing BUTTERFLY: frame 60 of 114 (52.6%)
Processing BUTTERFLY: frame 70 of 114 (61.4%)
Processing BUTTERFLY: frame 80 of 114 (70.2%)
Processing BUTTERFLY: frame 90 of 114 (78.9%)
Processing BUTTERFLY: frame 100 of 114 (87.7%)
Processing BUTTERFLY: frame 110 of 114 (96.5%)
Completed processing BUTTERFLY! Output saved to: /content/drive/MyDrive/sign_language_project/processed_videos/BUTTERFLY_landmarks.mp4

Processing sign: CAT2
Input video: /content/drive/MyDrive/sign_language_project/THIWANKA 2/CAT2/VID-20241023-WA0001 - Trim.mp4
Processing CAT2: frame 10 of 51 (19.6%

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm

class LandmarkDatasetCreator:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )

        # Define column names for the dataset
        self.landmark_columns = []
        for landmark in ['WRIST'] + [name.split('_')[0] for name in HAND_LANDMARKS_DICT.keys()]:
            self.landmark_columns.extend([f'{landmark}_x', f'{landmark}_y', f'{landmark}_z'])

    def extract_landmarks(self, frame):
        """Extract hand landmarks from a single frame"""
        # Convert BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the image
        results = self.hands.process(image)

        if not results.multi_hand_landmarks:
            return None

        # We'll use the first detected hand only for simplicity
        # You can modify this to handle multiple hands if needed
        landmarks = results.multi_hand_landmarks[0]

        # Extract coordinates
        coords = []
        for landmark in landmarks.landmark:
            coords.extend([landmark.x, landmark.y, landmark.z])

        return coords

    def process_video(self, video_path, sign_name):
        """Process a single video and extract landmarks from each frame"""
        cap = cv2.VideoCapture(video_path)
        frames_data = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            landmarks = self.extract_landmarks(frame)
            if landmarks is not None:
                frames_data.append(landmarks)

        cap.release()

        # Convert to DataFrame
        if frames_data:
            df = pd.DataFrame(frames_data, columns=self.landmark_columns)
            df['sign'] = sign_name
            return df

        return None

    def create_dataset(self, master_folder, output_path):
        """Create dataset from all videos in the master folder"""
        video_extensions = ('.mp4', '.avi', '.mov', '.mkv')
        all_data = []

        # Get all folders (signs)
        folders = [f for f in Path(master_folder).iterdir() if f.is_dir()]

        for folder in tqdm(folders, desc="Processing signs"):
            sign_name = folder.name
            print(f"\nProcessing sign: {sign_name}")

            # Get all videos for this sign
            videos = [f for f in folder.iterdir() if f.suffix.lower() in video_extensions]

            for video_path in tqdm(videos, desc="Processing videos", leave=False):
                print(f"Processing video: {video_path}")
                try:
                    df = self.process_video(str(video_path), sign_name)
                    if df is not None:
                        all_data.append(df)
                except Exception as e:
                    print(f"Error processing {video_path}: {str(e)}")

        # Combine all data
        if all_data:
            final_df = pd.concat(all_data, ignore_index=True)

            # Add statistical features
            print("Calculating statistical features...")
            for coord in ['x', 'y', 'z']:
                coord_cols = [col for col in final_df.columns if col.endswith(f'_{coord}')]
                final_df[f'mean_{coord}'] = final_df[coord_cols].mean(axis=1)
                final_df[f'std_{coord}'] = final_df[coord_cols].std(axis=1)
                final_df[f'max_{coord}'] = final_df[coord_cols].max(axis=1)
                final_df[f'min_{coord}'] = final_df[coord_cols].min(axis=1)

            # Save to CSV
            print(f"Saving dataset to {output_path}")
            final_df.to_csv(output_path, index=False)

            # Print dataset statistics
            print("\nDataset Statistics:")
            print(f"Total samples: {len(final_df)}")
            print("\nSamples per sign:")
            print(final_df['sign'].value_counts())

            return final_df

        return None

def main():
    # Define paths
    master_folder = "/content/drive/MyDrive/sign_language_project/THIWANKA 2/"
    output_path = "/content/drive/MyDrive/sign_language_project/hand_landmarks_dataset.csv"

    # Create dataset
    creator = LandmarkDatasetCreator()
    dataset = creator.create_dataset(master_folder, output_path)

    if dataset is not None:
        # Display sample of the dataset
        print("\nDataset Preview:")
        print(dataset.head())

        # Display feature statistics
        print("\nFeature Statistics:")
        print(dataset.describe())
    else:
        print("No data was processed successfully.")

if __name__ == "__main__":
    main()

Processing signs:   0%|          | 0/17 [00:00<?, ?it/s]


Processing sign: BUTTERFLY



Processing videos:   0%|          | 0/10 [00:00<?, ?it/s]

Processing video: /content/drive/MyDrive/sign_language_project/THIWANKA 2/BUTTERFLY/WhatsApp Video 2024-10-23 at 13.03.12_9c7463b6 - Trim.mp4



Processing videos:  10%|█         | 1/10 [00:09<01:28,  9.89s/it]

...

Processing video: /content/drive/MyDrive/sign_language_project/THIWANKA 2/triangle/WhatsApp Video 2024-10-23 at 20.46.08_0d24bb99 - Trim.mp4



Processing videos:  90%|█████████ | 9/10 [01:16<00:08,  8.10s/it]

Processing video: /content/drive/MyDrive/sign_language_project/THIWANKA 2/triangle/WhatsApp Video 2024-10-23 at 20.46.08_5a99bb8a - Trim.mp4



Processing signs: 100%|██████████| 17/17 [18:18<00:00, 64.62s/it]


Calculating statistical features...
Saving dataset to /content/drive/MyDrive/sign_language_project/hand_landmarks_dataset.csv

Dataset Statistics:
Total samples: 15134

Samples per sign:
sign
triangle       1263
square         1257
rectangle      1218
elephant       1125
CIRCLE         1114
parrot         1080
apple           986
banana2         900
pomegranate     881
CAT 1           854
pineapple       715
CAT2            692
BUTTERFLY       685
DOG             672
DOG2            650
mango           642
banana 1        400
Name: count, dtype: int64

Dataset Preview:
    WRIST_x   WRIST_y       WRIST_z   THUMB_x   THUMB_y   THUMB_z   THUMB_x  \
0  0.560414  0.711274  6.165005e-07  0.557226  0.662715 -0.019312  0.526104   
1  0.563585  0.712663  5.106817e-07  0.555333  0.661592 -0.011649  0.522962   
2  0.560852  0.712640  5.008951e-07  0.552608  0.661145 -0.011478  0.519723   
3  0.558943  0.710774  4.907292e-07  0.552418  0.659484 -0.010867  0.518417   
4  0.559112  0.708538  4.8472

In [ ]:
df = pd.read_csv(r"/content/drive/MyDrive/sign_language_project/hand_landmarks_dataset.csv")

In [ ]:
df.columns

Index(['WRIST_x', 'WRIST_y', 'WRIST_z', 'THUMB_x', 'THUMB_y', 'THUMB_z',
       'THUMB_x.1', 'THUMB_y.1', 'THUMB_z.1', 'THUMB_x.2', 'THUMB_y.2',
       'THUMB_z.2', 'THUMB_x.3', 'THUMB_y.3', 'THUMB_z.3', 'INDEX_x',
       'INDEX_y', 'INDEX_z', 'INDEX_x.1', 'INDEX_y.1', 'INDEX_z.1',
       'INDEX_x.2', 'INDEX_y.2', 'INDEX_z.2', 'INDEX_x.3', 'INDEX_y.3',
       'INDEX_z.3', 'MIDDLE_x', 'MIDDLE_y', 'MIDDLE_z', 'MIDDLE_x.1',
       'MIDDLE_y.1', 'MIDDLE_z.1', 'MIDDLE_x.2', 'MIDDLE_y.2', 'MIDDLE_z.2',
       'MIDDLE_x.3', 'MIDDLE_y.3', 'MIDDLE_z.3', 'RING_x', 'RING_y', 'RING_z',
       'RING_x.1', 'RING_y.1', 'RING_z.1', 'RING_x.2', 'RING_y.2', 'RING_z.2',
       'RING_x.3', 'RING_y.3', 'RING_z.3', 'PINKY_x', 'PINKY_y', 'PINKY_z',
       'PINKY_x.1', 'PINKY_y.1', 'PINKY_z.1', 'PINKY_x.2', 'PINKY_y.2',
       'PINKY_z.2', 'PINKY_x.3', 'PINKY_y.3', 'PINKY_z.3', 'sign', 'mean_x',
       'std_x', 'max_x', 'min_x', 'mean_y', 'std_y', 'max_y', 'min_y',
       'mean_z', 'std_z', 'max_z', 'min_z'],

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

class SignLanguageRandomForest:
    def __init__(self, n_estimators=100, max_depth=None):
        self.scaler = StandardScaler()
        self.label_encoder = LabelEncoder()
        self.model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            random_state=42,
            n_jobs=-1
        )

    def prepare_data(self, data_path):
        """Load and prepare the dataset"""
        print("Loading and preparing data...")

        # Load dataset
        self.df = pd.read_csv(data_path)  # Store the original dataframe

        # Split features and target
        X = self.df.drop('sign', axis=1)
        y = self.df['sign']

        # Store feature names
        self.feature_names = X.columns.tolist()

        # Encode labels
        y_encoded = self.label_encoder.fit_transform(y)

        # Scale features
        X_scaled = self.scaler.fit_transform(X)

        return X_scaled, y_encoded

    def train_and_evaluate(self, X, y, test_size=0.2):
        """Train the model and evaluate its performance"""
        # Create DataFrame with scaled features to maintain column names
        X_df = pd.DataFrame(X, columns=self.feature_names)

        # Split data
        X_train, X_test, y_train, y_test = train_test_split(
            X_df, y, test_size=test_size, random_state=42, stratify=y
        )

        print("Training Random Forest model...")
        # Train model
        self.model.fit(X_train, y_train)

        # Get training and test scores
        train_score = self.model.score(X_train, y_train)
        test_score = self.model.score(X_test, y_test)

        print(f"\nTraining accuracy: {train_score:.4f}")
        print(f"Test accuracy: {test_score:.4f}")

        # Make predictions
        y_pred = self.model.predict(X_test)

        # Print classification report
        print("\nClassification Report:")
        print(classification_report(
            y_test,
            y_pred,
            target_names=self.label_encoder.classes_
        ))

        # Plot confusion matrix
        self.plot_confusion_matrix(y_test, y_pred)

        # Plot feature importance
        self.plot_feature_importance()

        return X_test, y_test, y_pred

    def predict_new(self, X):
        """Make predictions on new data"""
        # Ensure X is a DataFrame with correct feature names
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        predictions = self.model.predict(X)
        return self.label_encoder.inverse_transform(predictions)

    def plot_confusion_matrix(self, y_true, y_pred):
        """Plot confusion matrix"""
        plt.figure(figsize=(12, 8))
        cm = confusion_matrix(y_true, y_pred)
        sns.heatmap(
            cm,
            annot=True,
            fmt='d',
            cmap='Blues',
            xticklabels=self.label_encoder.classes_,
            yticklabels=self.label_encoder.classes_
        )
        plt.title('Confusion Matrix - Random Forest')
        plt.xlabel('Predicted')
        plt.ylabel('True')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('rf_confusion_matrix.png')
        plt.close()

    def plot_feature_importance(self):
        """Plot feature importance"""
        importance = self.model.feature_importances_
        feat_importance = pd.DataFrame({
            'feature': self.feature_names,
            'importance': importance
        })
        feat_importance = feat_importance.sort_values(
            'importance',
            ascending=False
        ).head(20)

        plt.figure(figsize=(12, 6))
        sns.barplot(
            x='importance',
            y='feature',
            data=feat_importance
        )
        plt.title('Top 20 Most Important Features - Random Forest')
        plt.tight_layout()
        plt.savefig('rf_feature_importance.png')
        plt.close()

def main():
    # Define paths
    data_path = "/content/drive/MyDrive/sign_language_project/hand_landmarks_dataset.csv"
    model_path = "/content/drive/MyDrive/sign_language_project/rf_sign_language_model.joblib"
    scaler_path = "/content/drive/MyDrive/sign_language_project/rf_sign_language_scaler.joblib"

    # Initialize classifier
    classifier = SignLanguageRandomForest(
        n_estimators=100,
        max_depth=10
    )

    # Prepare data
    X, y = classifier.prepare_data(data_path)

    # Train and evaluate
    X_test, y_test, y_pred = classifier.train_and_evaluate(X, y)

    # Save model
    joblib.dump((classifier.model, classifier.scaler, classifier.label_encoder, classifier.feature_names),
                model_path)
    print(f"\nModel and associated components saved to {model_path}")

    # Test prediction on a few samples
    print("\nSample Predictions:")
    sample_size = 5
    sample_indices = np.random.choice(len(X_test), sample_size)

    # Create proper DataFrame for sample predictions
    X_sample = X_test.iloc[sample_indices]
    y_true = y_test[sample_indices]

    predictions = classifier.predict_new(X_sample)
    true_labels = classifier.label_encoder.inverse_transform(y_true)

    print("\nDetailed Sample Predictions:")
    for true, pred in zip(true_labels, predictions):
        print(f"True: {true}, Predicted: {pred}")
        if true != pred:
            print(f"WARNING: Misclassification detected!")

if __name__ == "__main__":
    main()

Loading and preparing data...
Training Random Forest model...

Training accuracy: 0.9817
Test accuracy: 0.9716

Classification Report:
              precision    recall  f1-score   support

   BUTTERFLY       1.00      0.99      1.00       137
       CAT 1       1.00      0.97      0.99       171
        CAT2       0.94      0.96      0.95       138
      CIRCLE       1.00      1.00      1.00       223
         DOG       1.00      0.99      0.99       134
        DOG2       1.00      0.99      1.00       130
       apple       0.96      0.99      0.98       197
    banana 1       1.00      1.00      1.00        80
     banana2       0.92      0.96      0.94       180
    elephant       1.00      1.00      1.00       225
       mango       0.98      0.99      0.99       128
      parrot       1.00      1.00      1.00       216
   pineapple       0.94      0.97      0.95       143
 pomegranate       1.00      1.00      1.00       176
   rectangle       0.97      0.91      0.94       244


# Model Inferece


In [7]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import joblib
from pathlib import Path

class SignLanguageInference:
    def __init__(self, model_path):
        # Load MediaPipe
        self.mp_hands = mp.solutions.hands
        self.hands = self.mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )

        # Load the trained model and components
        self.model, self.scaler, self.label_encoder, self.feature_names = joblib.load(model_path)

    def extract_landmarks(self, frame):
        """Extract hand landmarks from a single frame"""
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.hands.process(image)

        if not results.multi_hand_landmarks:
            return None

        landmarks = results.multi_hand_landmarks[0]
        coords = []
        for landmark in landmarks.landmark:
            coords.extend([landmark.x, landmark.y, landmark.z])

        return coords

    def process_video(self, video_path):
        """Process video and return predictions"""
        cap = cv2.VideoCapture(video_path)
        frames_data = []

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            landmarks = self.extract_landmarks(frame)
            if landmarks is not None:
                frames_data.append(landmarks)

        cap.release()

        if not frames_data:
            return None

        # Create DataFrame with landmarks
        df = pd.DataFrame(frames_data, columns=self.feature_names[:-12])  # Exclude statistical features

        # Calculate statistical features
        for coord in ['x', 'y', 'z']:
            coord_cols = [col for col in df.columns if col.endswith(f'_{coord}')]
            df[f'mean_{coord}'] = df[coord_cols].mean(axis=1)
            df[f'std_{coord}'] = df[coord_cols].std(axis=1)
            df[f'max_{coord}'] = df[coord_cols].max(axis=1)
            df[f'min_{coord}'] = df[coord_cols].min(axis=1)

        # Scale features
        X_scaled = self.scaler.transform(df)

        # Make predictions
        predictions = self.model.predict(X_scaled)
        return self.label_encoder.inverse_transform(predictions)

    def predict(self, video_path):
        """Predict sign language from video"""
        predictions = self.process_video(video_path)
        if predictions is None:
            return "No hand landmarks detected"

        # Return most common prediction
        unique_preds, counts = np.unique(predictions, return_counts=True)
        return unique_preds[np.argmax(counts)]

def main():
    model_path = "/content/drive/MyDrive/sign_language_project/rf_sign_language_model.joblib"
    video_path = "/content/drive/MyDrive/sign_language_project/THIWANKA 2/Wrong-video/Wrong_video.mp4"

    inference = SignLanguageInference(model_path)
    prediction = inference.predict(video_path)
    print(f"Predicted sign: {prediction}")


main()

Predicted sign: No hand landmarks detected
